In [ ]:
%matplotlib inline
from IPython.display import clear_output
from pytesseract import image_to_string

from srbot import *

In [ ]:
#Bow fletcher (falador west bank)

log = load_image('willow_logs.png')
knife = load_image('knife.png')

fletch_loc = np.asarray([254,398])

target()
total_inventories = 0
last_fletch = mark_time()
logins = 0
while True:
    client = get_client()
    if len(find_bitmap(loginscreen,client)) > 0:
        logins = logins + 1
        if logins > 10:
            raise RuntimeError('Too many logins! Bailing out before we get fishy!')
        login()
        continue
    if mark_time()-last_fletch > 10*60:
        raise RuntimeError('Took more than 10min to complete a bow, something is wrong.')
    
    inventory = get_inventory()
    found = find_best_bitmap(log,inventory,tol=0.15,mode='dist')
    if len(found) > 0: #fletch em
        print('time to fletch')
        mainscreen = get_mainscreen()
        if len(find_bitmap(bank_window,mainscreen)) > 0:
            click_mouse(486,44)
            sleep(1.0)
        k = find_best_bitmap(knife, inventory, mode='dist', tol=0.02)
        l = find_best_bitmap(log, inventory, mode='dist', tol=0.2)
        print(len(k),len(l))
        if len(k) > 0 and len(l) > 0:
            np.random.shuffle(k)
            np.random.shuffle(l)
            click_mouse(*(k[0]+[ivxs,ivys]))
            sleep(1.0)
            click_mouse(*(l[0]+[ivxs,ivys]))
            sleep(1.0)
            click_mouse(*fletch_loc,left=False)
            sleep(1.0)
            click_mouse(*(fletch_loc+[55-76,477-405]))
            sleep(1.0)
            send_keys('9999')
            sleep(1.0)
            send_keys('\n')
            i = 0
            last_len = len(l)
            while True:
                inventory = get_inventory()
                new_len = len(find_best_bitmap(log, inventory,mode='dist', tol=0.2))
                if new_len == 0:
                    break
                if last_len != new_len:
                    last_len = new_len
                    last_fletch = mark_time()
                    i = 0
                i=i+1
                if i > 4:
                    print('timed out')
                    break
                sleep(0.5)
        else:
            print('missing knife or logs')
    else: #bank
        print('doing bank stuff')
        mainscreen = get_mainscreen()
        if len(find_bitmap(bank_window,mainscreen)) == 0:
            pa = find_colors([74,70,70],mainscreen,tol=0.02,mode='hsl')
            pb = find_colors([118,96,68],mainscreen,tol=0.02,mode='hsl')
            points = filter_near(pa,pb,20)
            points = points[np.argsort(np.sum(np.square(points-[msxc-msxs,msyc-msys]),axis=1))]
            for point in points:
                click_mouse(*point,left=False)
                sleep(0.05)
                use = find_bitmap(use_booth,get_mainscreen())
                if len(use) > 0:
                    click_mouse(*(use[0]+[10,10]))
                    flag_wait()
                    break
            continue
        deposit_all(ignore_first=True)
        mainscreen = get_mainscreen()
        found = find_best_bitmap(log,mainscreen,tol=0.35,mode='dist')
        print('bank logs',len(found))
        if len(found) > 0:
            point = found[np.random.randint(len(found))]
            click_mouse(*(point+[msxs,msys]),left=False)
            sleep(1.0)
            click_mouse(*(point+[msxs,msys]+[0,87]))
            sleep(1.0)
            send_keys('27\n')
            sleep(1.0)
        else:
            print('out of supplies!')
            break
        click_mouse(486,44)
        sleep(1.0)